In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
PINECONE_API_KEY = 'ca7b56e2-0178-4183-a375-06b9ddc60204'

In [ ]:
!pip install -qU langchain==0.1.1
!pip install -qU langchain-community==0.0.13
!pip install -qU tiktoken==0.4.0
!pip install -qU pinecone-client==3.0.0
!pip install -qU pinecone-datasets==0.7.0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
keras-cv 0.8.2 requires keras-core, which is not installed.
keras-nlp 0.8.1 requires keras-core, which is not installed.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
cudf 23.8.0 requires cuda-python<12.0a0,>=11.7.1, but you have cuda-python 12.3.0 which is incompatible.
cudf 23.8.0 requires pandas<1.6.0dev0,>=1.3, but you have pandas 2.1.4 which is incompatible.
cudf 23.8.0 requires protobuf<5,>=4.21, but you have protobuf 3.20.3 which is incompatible.
cuml 23.8.0 r

In [12]:
import pinecone_datasets
dataset = pinecone_datasets.load_dataset('wikipedia-simple-text-embedding-ada-002-100K')

In [14]:
# drop metadata column and renamed blob to metadata
dataset.documents.drop(['metadata'], axis=1, inplace=True)
dataset.documents.rename(columns={'blob': 'metadata'}, inplace=True)


# sample 30k documents
dataset.documents.drop(dataset.documents.index[30_000:], inplace=True)

In [ ]:
from pinecone import Pinecone, PodSpec

pc = Pinecone(api_key=PINECONE_API_KEY)

In [ ]:
# Check for and delete index if already exists
index_name = 'langchain-retrieval-augmentation-fast'

In [49]:
if index_name in pc.list_indexes().names():
    pc.delete_index(index_name)

vectorstore = pc.create_index(
  name=index_name,
  dimension=1536,
  metric="cosine",
  spec=PodSpec(
    environment="gcp-starter"
  )
)

In [50]:
vectorstore = pc.Index(index_name)
vectorstore.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [51]:
for batch in dataset.iter_documents(batch_size=100):
    index.upsert(batch)

In [ ]:
vectorstore = pc.Index(index_name)
vectorstore.describe_index_stats()

In [73]:
pc.list_indexes()

{'indexes': [{'dimension': 1536,
              'host': 'langchain-retrieval-augmentation-fast-y8ilu5k.svc.gcp-starter.pinecone.io',
              'metric': 'cosine',
              'name': 'langchain-retrieval-augmentation-fast',
              'spec': {'pod': {'environment': 'gcp-starter',
                               'pod_type': 'starter',
                               'pods': 1,
                               'replicas': 1,
                               'shards': 1}},
              'status': {'ready': True, 'state': 'Ready'}}]}

In [75]:
from transformers import AutoModel, AutoTokenizer

# Assuming the model name is 'sentence-transformers/all-mpnet-base-v2'
model_name = "sentence-transformers/all-mpnet-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [79]:
def get_concatenated_embeddings(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
    
    # Concatenate embeddings from two layers
    concatenated_embeddings = torch.cat(outputs.hidden_states[-2:], dim=-1)
    
    # Use the mean or another aggregation method based on your requirements
    aggregated_embedding = concatenated_embeddings.mean(dim=1)
    
    return aggregated_embedding.numpy().tolist()[0]

query_text = "FreeBirds Crew"
query_embedding = get_concatenated_embeddings(query_text, tokenizer, model)
len(query_embedding)

1536

In [ ]:
# Index the text embedding in Pinecone
embedding = [{'id': "01", 'values': query_embedding}]
vectorstore.upsert(query_embedding)

In [89]:
results = vectorstore.query(vector=embedding, top_k=5)
results

{'matches': [{'id': '01', 'score': 1.00593686, 'values': []},
             {'id': '17651-9', 'score': 0.0617223866, 'values': []},
             {'id': '39669-0', 'score': 0.0613081716, 'values': []},
             {'id': '9589-17', 'score': 0.0588944554, 'values': []},
             {'id': '10754-4', 'score': 0.0565854795, 'values': []}],
 'namespace': '',
 'usage': {'read_units': 5}}

In [115]:
query_text = "What are Power Rangers?"
query_embedding = get_concatenated_embeddings(query_text, tokenizer, model)
len(query_embedding)

1536

In [125]:
import json
results = vectorstore.query(vector=query_embedding, top_k=1, include_metadata=True)
for result in results["matches"]:
    print(result['score']*100,'%')
    print(result['metadata']['text'])

4.27648835 %
The week after Aaliyah's death, her third studio album, Aaliyah, rose from number 19 to number one on the Billboard 200. "Rock the Boat" was released as a posthumous single. The music video premiered on BET's Access Granted; it became the most viewed and highest rated episode in the history of the show. The song peaked at number 14 on the Billboard Hot 100 and number two on the Billboard Hot R&B/Hip-Hop Songs chart. It was also included on the Now That's What I Call Music! 8 compilation series; a portion of the album's profits was donated to the Aaliyah Memorial Fund. The following two singles from Aaliyah, "More than a Woman" and "I Care 4 U", peaked within the top 25 of the Billboard Hot 100. The album was certified double Platinum by the RIAA and sold 2.95 million copies in the United States. "More than a Woman" reached number one in the UK singles chart making Aaliyah the first deceased artist to reach number one in the UK single chart. "More than a Woman" was replaced

In [1]:
from langchain.vectorstores import Pinecone
query = "What is Schizophrenia?"
vectorstore = Pinecone()

ModuleNotFoundError: No module named 'langchain'

In [139]:
vectorstore